In [1]:
import numpy as np 
import pandas as pd 
import os 
from transformers import *
import seaborn as sns
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, TensorDataset, DataLoader, RandomSampler, SequentialSampler 
import time
from datetime import datetime 
from sklearn.model_selection import StratifiedKFold
from tqdm.auto import tqdm
import torch
import torch.nn as nn 
import torch.nn.functional as F
from sklearn.utils.class_weight import compute_class_weight 
import seaborn as sns 

/opt/conda/lib/python3.8/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

tokenizer = AutoTokenizer.from_pretrained("monologg/kobigbird-bert-base") 

class MeanPooling(nn.Module): 
    def __init__(self): 
        super(MeanPooling, self).__init__() 
    def forward(self, last_hidden_state, attention_masks): 
        input_mask_expanded = attention_masks.unsqueeze(-1).expand(last_hidden_state.size()).float() 
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1) 
        sum_mask = input_mask_expanded.sum(1) 
        sum_mask = torch.clamp(sum_mask, min=1e-9) 
        mean_embeddings = sum_embeddings / sum_mask 
        return mean_embeddings 
    
class MultiSampleDropout(nn.Module): 
    def __init__(self, max_dropout_rate, num_samples, classifier):
        super(MultiSampleDropout, self).__init__()
        self.dropout = nn.Dropout
        self.classifier = classifier
        self.max_dropout_rate = max_dropout_rate
        self.num_samples = num_samples
    def forward(self, out):
        return torch.mean(torch.stack([self.classifier(self.dropout(p=rate)(out)) for _, rate in enumerate(np.linspace(0, self.max_dropout_rate, self.num_samples))], dim=0), dim=0)

class ChatGPTDetector(nn.Module): 
    def __init__(self, num_classes=2): 
        super(ChatGPTDetector, self).__init__() 
        self.num_classes = num_classes 
        self.config = AutoConfig.from_pretrained("monologg/kobigbird-bert-base")
        self.lm = AutoModel.from_pretrained("monologg/kobigbird-bert-base") 
        self.mean_pooler = MeanPooling() 
        self.fc = nn.Linear(self.config.hidden_size, self.num_classes) 
        self._init_weights(self.fc) 
        self.multi_dropout = MultiSampleDropout(0.2, 8, self.fc) 
    def _init_weights(self, module):
        if isinstance(module, nn.Linear): 
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range) 
            if module.bias is not None: 
                module.bias.data.zero_() 
    def forward(self, input_ids, attn_masks):
        x = self.lm(input_ids, attn_masks)[0]
        x = self.mean_pooler(x, attn_masks) 
        x = self.multi_dropout(x)
        return x 

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--monologg--kobigbird-bert-base/snapshots/ceacda477e20abef2c929adfa4a07c6f811323be/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--monologg--kobigbird-bert-base/snapshots/ceacda477e20abef2c929adfa4a07c6f811323be/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--monologg--kobigbird-bert-base/snapshots/ceacda477e20abef2c929adfa4a07c6f811323be/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--monologg--kobigbird-bert-base/snapshots/ceacda477e20abef2c929adfa4a07c6f811323be/tokenizer_config.json


In [7]:
model = ChatGPTDetector()
checkpoint = torch.load("KR_ChatGPT_Detector_v1.pt")
model.load_state_dict(checkpoint) 
model.to(device)
model.eval()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--monologg--kobigbird-bert-base/snapshots/ceacda477e20abef2c929adfa4a07c6f811323be/config.json
Model config BigBirdConfig {
  "_name_or_path": "monologg/kobigbird-bert-base",
  "architectures": [
    "BigBirdForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "attention_type": "block_sparse",
  "block_size": 64,
  "bos_token_id": 5,
  "classifier_dropout": null,
  "eos_token_id": 6,
  "gradient_checkpointing": false,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 4096,
  "model_type": "big_bird",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_random_blocks": 3,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "rescale_embeddings": false,
  "sep_token_id": 3,
  "tokenizer_class": "BertTokenizer",
  "torch_dtype": "floa

ChatGPTDetector(
  (lm): BigBirdModel(
    (embeddings): BigBirdEmbeddings(
      (word_embeddings): Embedding(32500, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BigBirdEncoder(
      (layer): ModuleList(
        (0): BigBirdLayer(
          (attention): BigBirdAttention(
            (self): BigBirdBlockSparseAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): BigBirdSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (dropout):

In [14]:
# 실제 ChatGPT가 생성한 텍스트 
passage = """스마트 컨트랙트는 블록체인 기술에서 사용되는 프로그램 코드의 일종으로, 계약 조건을 자동으로 검증하고 실행하는 프로그램입니다. 스마트 컨트랙트는 블록체인 네트워크 상에서 실행되며, 블록체인에서 발생하는 모든 트랜잭션은 스마트 컨트랙트를 통해 처리됩니다.

스마트 컨트랙트는 조건과 실행 코드로 이루어져 있습니다. 예를 들어, A가 B에게 1,000달러를 지불해야하는 계약이 있다면, 이 계약의 조건을 스마트 컨트랙트로 작성하여 계약이 자동으로 실행되도록 할 수 있습니다. 이러한 스마트 컨트랙트는 자동으로 조건을 검증하고, 지정된 조건이 충족되었을 때 계약의 실행 코드를 실행하여 계약을 이행합니다. 이를 통해 계약 당사자는 서로를 신뢰하지 않아도 안전하게 거래를 진행할 수 있습니다.

스마트 컨트랙트는 블록체인에서 실행되기 때문에 모든 거래 내역이 투명하게 기록되며, 중개인이나 중앙 기관의 개입이 없기 때문에 거래 비용이 줄어듭니다. 또한 스마트 컨트랙트는 코드로 작성되기 때문에 자동화가 가능하며, 프로그램에 따른 비즈니스 로직을 실행하는 것이 가능합니다. 따라서 스마트 컨트랙트는 블록체인 기술의 핵심 기능 중 하나로, 분산형 애플리케이션(DApp) 개발 등에 활용되고 있습니다."""


encoded_input = tokenizer(passage, max_length=512, truncation=True, padding="max_length", return_tensors="pt").to(device) 
input_ids = encoded_input["input_ids"] 
attn_masks = encoded_input["attention_mask"] 

with torch.no_grad(): 
    output = model(input_ids, attn_masks) 
    pred_class = nn.Softmax()(output)[0]
    print(f"ChatGPT가 생성했을 확률: {pred_class[1]*100}%")
    


ChatGPT가 생성했을 확률: 99.98743438720703%


/tmp/ipykernel_6773/3078326761.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred_class = nn.Softmax()(output)[0]


In [29]:
bigbird = AutoModel.from_pretrained("monologg/kobigbird-bert-base")
bigbird_state_dict = bigbird.state_dict() 

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--monologg--kobigbird-bert-base/snapshots/ceacda477e20abef2c929adfa4a07c6f811323be/config.json
Model config BigBirdConfig {
  "_name_or_path": "monologg/kobigbird-bert-base",
  "architectures": [
    "BigBirdForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "attention_type": "block_sparse",
  "block_size": 64,
  "bos_token_id": 5,
  "classifier_dropout": null,
  "eos_token_id": 6,
  "gradient_checkpointing": false,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 4096,
  "model_type": "big_bird",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_random_blocks": 3,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "rescale_embeddings": false,
  "sep_token_id": 3,
  "tokenizer_class": "BertTokenizer",
  "torch_dtype": "floa

In [30]:
cnt = 0 
for key, value in bigbird_state_dict.items(): 
    check_key = "lm." + str(key) 
    if check_key in checkpoint.keys():
        bigbird_state_dict[key] = checkpoint[check_key] 
        cnt += 1 
        

In [32]:
bigbird.load_state_dict(bigbird_state_dict) 

<All keys matched successfully>

In [33]:
torch.save(bigbird, "pytorch_model.bin")